# xarray analysis of 2017 data
---

I wrote the original analysis notebooks to use the pandas library, but also want to conduct this analysis in xarray to get more comfortable with the library. 

In [8]:
import warnings

warnings.simplefilter("ignore", category=DeprecationWarning)

In [7]:
# Imports
import requests
import re
import os
import numpy as np
import xarray as xr
import pandas as pd

---
### Pull in data
I tried using the OOI `get_data` function and having it return xarray datasets instead of pandas dataframes, which worked, but I wasn't able to save the files as netCDF files due to an issue with the format of the fillvalues. I know there are ways to download the netCDF files directly from the data server but couldn't figure it out today. Instead, I'm just loading my csv files and converting them to xarray datasets, I hate this workaround but I really just want to move on. Come up with a better solution by Tuesday please! 

In [9]:
METBK_data = pd.read_csv('../../coastal_upwelling_output/metbk_data.csv')
profiler_data = pd.read_csv('../../coastal_upwelling_output/profiler_data.csv')
platform_data = pd.read_csv('../../coastal_upwelling_output/platform_data.csv')

In [12]:
METBK_ds = METBK_data.to_xarray()
METBK_ds = METBK_ds.swap_dims({'index':'time'})
METBK_ds

<xarray.Dataset>
Dimensions:                     (time: 485137)
Coordinates:
    index                       (time) int64 0 1 2 3 ... 485134 485135 485136
  * time                        (time) object '2017-01-01 00:00:30.432000000'...
Data variables:
    sea_surface_temperature     (time) float64 11.27 11.27 11.27 ... 12.05 12.05
    met_windavg_mag_corr_east   (time) float64 4.095 6.135 ... -1.228e+07
    met_windavg_mag_corr_north  (time) float64 -7.197 -5.29 ... -7.014e+06

In [13]:
profiler_ds = profiler_data.to_xarray()
profiler_ds = profiler_ds.swap_dims({'index':'time'})
profiler_ds

<xarray.Dataset>
Dimensions:                     (time: 21517483)
Coordinates:
    index                       (time) int64 0 1 2 ... 21517481 21517482
  * time                        (time) object '2017-01-01 00:00:00.317937152'...
Data variables:
    seawater_pressure           (time) float64 200.5 200.5 200.5 ... 55.55 55.67
    density                     (time) float64 1.027e+03 1.027e+03 ... 1.026e+03
    practical_salinity          (time) float64 33.89 33.89 33.89 ... 33.52 33.52
    seawater_temperature        (time) float64 7.786 7.786 7.786 ... 9.729 9.735
    corrected_dissolved_oxygen  (time) float64 106.6 106.6 106.6 ... 174.1 174.0

In [14]:
platform_ds = platform_data.to_xarray()
platform_ds = platform_ds.swap_dims({'index':'time'})
platform_ds

<xarray.Dataset>
Dimensions:               (time: 21413155)
Coordinates:
    index                 (time) int64 0 1 2 3 ... 21413152 21413153 21413154
  * time                  (time) object '2017-01-01 00:00:00.358334464' ... '...
Data variables:
    seawater_pressure     (time) float64 174.1 174.1 174.1 ... 105.8 105.6 105.6
    density               (time) float64 1.027e+03 1.027e+03 ... 1.027e+03
    practical_salinity    (time) float64 33.92 33.92 33.92 ... 33.8 33.8 33.8
    seawater_temperature  (time) float64 7.768 7.767 7.767 ... 8.838 8.838 8.839
    dissolved_oxygen      (time) float64 243.2 243.2 243.2 ... 141.1 141.0 141.1

Save these as netCDF just for practice.

In [15]:
METBK_ds.to_netcdf('../../coastal_upwelling_output/metbk_data_2017.nc')
profiler_ds.to_netcdf('../../coastal_upwelling_output/profiler_data_2017.nc')
platform_ds.to_netcdf('../../coastal_upwelling_output/platform_data_2017.nc')

Maintain the 

In [3]:
# Specify the variable(s) of interest
METBK_var = ['sea_surface_temperature', 'met_windavg_mag_corr_east', 'met_windavg_mag_corr_north']
profiler_var = ['seawater_pressure', 'density', 'practical_salinity', 'seawater_temperature', 'corrected_dissolved_oxygen']
platform_var = ['seawater_pressure', 'density', 'practical_salinity', 'seawater_temperature', 'dissolved_oxygen']

In [4]:
# Specify the units for each variable of interest
METBK_units = {'sea_surface_temperature': ['ºC'], 'met_windavg_mag_corr_east': ['m s-1'], 'met_windavg_mag_corr_north': ['m s-1']}
profiler_units = {'seawater_pressure': ['dbar'], 'density': ['kg m-3'], 'practical_salinity': ['1'], 'seawater_temperature': ['ºC'], 'corrected_dissolved_oxygen': ['µmol kg-1']}
platform_units = {'seawater_pressure': ['dbar'], 'density': ['kg m-3'], 'practical_salinity': ['1'], 'seawater_temperature': ['ºC'], 'dissolved_oxygen': ['µmol kg-1']}

In [17]:
METBK_ds.isnull().sum()

<xarray.Dataset>
Dimensions:                     ()
Data variables:
    sea_surface_temperature     int32 0
    met_windavg_mag_corr_east   int32 17981
    met_windavg_mag_corr_north  int32 17981

The wind velocities have some nulls, I'm not sure why. 

In [18]:
profiler_ds.isnull().sum()

<xarray.Dataset>
Dimensions:                     ()
Data variables:
    seawater_pressure           int32 0
    density                     int32 0
    practical_salinity          int32 0
    seawater_temperature        int32 0
    corrected_dissolved_oxygen  int32 0

In [19]:
platform_ds.isnull().sum()

<xarray.Dataset>
Dimensions:               ()
Data variables:
    seawater_pressure     int32 0
    density               int32 0
    practical_salinity    int32 0
    seawater_temperature  int32 0
    dissolved_oxygen      int32 0

The profiler and platform data are null free, excellent.

---
### METBK data

First, look at the northward wind velocity data.

In [26]:
METBK_ds.var

<bound method ImplementsDatasetReduce._reduce_method.<locals>.wrapped_func of <xarray.Dataset>
Dimensions:                     (time: 485137)
Coordinates:
    index                       (time) int64 0 1 2 3 ... 485134 485135 485136
  * time                        (time) object '2017-01-01 00:00:30.432000000'...
Data variables:
    sea_surface_temperature     (time) float64 11.27 11.27 11.27 ... 12.05 12.05
    met_windavg_mag_corr_east   (time) float64 4.095 6.135 ... -1.228e+07
    met_windavg_mag_corr_north  (time) float64 -7.197 -5.29 ... -7.014e+06>

In [ ]:
# Convert time to datetime


In [33]:
ds = xr.open_dataset('https://opendap.oceanobservatories.org/thredds/catalog/ooi/deryag@uw.edu/20210422T030752259Z-CE04OSSM-SBD11-06-METBKA000-recovered_host-metbk_a_dcl_instrument_recovered/catalog.html') #decode_times=False)

OSError: [Errno -90] NetCDF: file not found: b'https://opendap.oceanobservatories.org/thredds/catalog/ooi/deryag@uw.edu/20210422T030752259Z-CE04OSSM-SBD11-06-METBKA000-recovered_host-metbk_a_dcl_instrument_recovered/catalog.html'

In [29]:
ds


<xarray.Dataset>
Dimensions:  (T: 1420, X: 1405, Y: 621)
Coordinates:
  * Y        (Y) float32 49.92 49.88 49.83 49.79 ... 24.21 24.17 24.12 24.08
  * X        (X) float32 -125.0 -125.0 -124.9 -124.9 ... -66.58 -66.54 -66.5
  * T        (T) float32 -779.5 -778.5 -777.5 -776.5 ... 636.5 637.5 638.5 639.5
Data variables:
    ppt      (T, Y, X) float64 ...
    tdmean   (T, Y, X) float64 ...
    tmax     (T, Y, X) float64 ...
    tmin     (T, Y, X) float64 ...
Attributes:
    Conventions:  IRIDL

In [36]:
# !pip install git+https://github.com/seagrinch/ooilab.git

  Cloning https://github.com/seagrinch/ooilab.git to c:\users\derya\appdata\local\temp\pip-req-build-dgax3_8h
  Created wheel for ooilab: filename=ooilab-0.2-py3-none-any.whl size=2590 sha256=25c6e94d1f68f190585206a09d38907d3a5c71c712f59e3b35f70577db429de3
  Stored in directory: C:\Users\Derya\AppData\Local\Temp\pip-ephem-wheel-cache-9r8_8x56\wheels\7b\26\19\c4ea5b81a92c0dd1969bf0d89c4208414ba9675999df27c8c7
Successfully built ooilab


  Running command git clone -q https://github.com/seagrinch/ooilab.git 'C:\Users\Derya\AppData\Local\Temp\pip-req-build-dgax3_8h'


In [42]:
# enter your OOI API username and token, and directory where output .csv files are saved
API_USERNAME = 'OOIAPI-U6ZIZ5UNB1LIMA'
API_TOKEN = '582X6TXS4S'
save_dir = '/Users/Derya/Desktop/Acoustics/Code/output/august15/'

In [43]:
import ooilab

In [44]:
url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/deryag@uw.edu/20210422T030752259Z-CE04OSSM-SBD11-06-METBKA000-recovered_host-metbk_a_dcl_instrument_recovered/catalog.html'

In [47]:
flist = ooilab.get_filelist(url)
flist

['https://opendap.oceanobservatories.org/thredds/dodsC\\ooi/deryag@uw.edu/20210422T030752259Z-CE04OSSM-SBD11-06-METBKA000-recovered_host-metbk_a_dcl_instrument_recovered/deployment0003_CE04OSSM-SBD11-06-METBKA000-recovered_host-metbk_a_dcl_instrument_recovered_20170101T000030.432000-20170421T150846.868000.nc#fillmismatch',
 'https://opendap.oceanobservatories.org/thredds/dodsC\\ooi/deryag@uw.edu/20210422T030752259Z-CE04OSSM-SBD11-06-METBKA000-recovered_host-metbk_a_dcl_instrument_recovered/deployment0004_CE04OSSM-SBD11-06-METBKA000-recovered_host-metbk_a_dcl_instrument_recovered_20170421T022518.003000-20171013T154805.602000.nc#fillmismatch',
 'https://opendap.oceanobservatories.org/thredds/dodsC\\ooi/deryag@uw.edu/20210422T030752259Z-CE04OSSM-SBD11-06-METBKA000-recovered_host-metbk_a_dcl_instrument_recovered/deployment0005_CE04OSSM-SBD11-06-METBKA000-recovered_host-metbk_a_dcl_instrument_recovered_20171011T234120.099000-20171201T183226.453000.nc#fillmismatch']

In [48]:
data = xr.open_mfdataset(flist).swap_dims({'obs': 'time'}).sortby('time')

OSError: [Errno -77] NetCDF: Access failure: b'https://opendap.oceanobservatories.org/thredds/dodsC\\ooi/deryag@uw.edu/20210422T030752259Z-CE04OSSM-SBD11-06-METBKA000-recovered_host-metbk_a_dcl_instrument_recovered/deployment0003_CE04OSSM-SBD11-06-METBKA000-recovered_host-metbk_a_dcl_instrument_recovered_20170101T000030.432000-20170421T150846.868000.nc#fillmismatch'

In [52]:
# import necessary functions

import numpy as np
from numpy import datetime64 as dt64, timedelta64 as td64
import pickle

import sys, os
import shutil
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt

from netCDF4 import Dataset, num2date, date2num 
from datetime import datetime, timedelta

from yodapy.datasources import OOI
ooi = OOI()

# Local application/library specific imports
from golive_library import GoliveLibrary as g

from yodapy.utils.creds import set_credentials_file

C:\Users\Derya\anaconda3\envs\echospace\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ImportError: cannot import name 'ConvertEK60' from 'echopype.convert' (C:\Users\Derya\anaconda3\envs\echospace\lib\site-packages\echopype\convert\__init__.py)

In [51]:
pip install git+https://github.com/cormorack/yodapy.git

C:\Users\Derya\anaconda3\envs\echospace\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  Cloning https://github.com/cormorack/yodapy.git to c:\users\derya\appdata\local\temp\pip-req-build-oe4mf1x3
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requests-oauthlib>=0.7.0 in c:\users\derya\anaconda3\envs\echospace\lib\site-packages (from google-auth-oauthlib>=0.4.1->gspread->yodapy==0.3.0b0+5.g95ce171) (1.3.0)


  Running command git clone -q https://github.com/cormorack/yodapy.git 'C:\Users\Derya\AppData\Local\Temp\pip-req-build-oe4mf1x3'
